In [37]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os
load_dotenv()

True

In [49]:
gpt_turbo = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [50]:
import requests
url = 'https://public-api.meteofrance.fr/public/DPVigilance/v1/textesvigilance/encours'
headers = {
    'accept': '*/*',
    'apikey': os.environ['VIGILENCE_API_KEY']
}
response = requests.get(url, headers=headers)
data = response.json()
data

{'product': {'warning_type': 'vigilance',
  'type_cdp': 'cdp_textes',
  'version_vigilance': 'V6',
  'version_cdp': '1.0.0',
  'domain_id': 'FRA',
  'domain_name': 'France',
  'update_time': '2024-04-08T08:00:31Z',
  'text_bloc_items': [{'domain_id': 'FRA',
    'domain_name': 'France',
    'bloc_title': 'Bulletin de Vigilance météo national',
    'bloc_id': 'BULLETIN_NATIONAL',
    'bloc_items': [{'id': 'NAT_SITUATION_PAYS',
      'type_name': 'Situation Actuelle',
      'type_group': 'SITUATION',
      'text_items': [{'type_code': 'SITUATION_PAYS_SITUATION_GÉNÉRALE_NATIONALE',
        'hazard_code': None,
        'hazard_name': 'tous aléas',
        'term_items': [{'term_names': 'J+J1',
          'start_time': '2024-04-08T08:00:00Z',
          'end_time': '2024-04-09T22:00:00Z',
          'risk_name': 'Orange',
          'risk_code': '3',
          'risk_color': '#f7a401',
          'risk_level': 2,
          'subdivision_text': [{'underline_text': '',
            'text': ['Faits nouv

In [59]:
data['product'].keys()

dict_keys(['warning_type', 'type_cdp', 'version_vigilance', 'version_cdp', 'domain_id', 'domain_name', 'update_time', 'text_bloc_items'])

In [63]:
data['product']['text_bloc_items'][0]

{'domain_id': 'FRA',
 'domain_name': 'France',
 'bloc_title': 'Bulletin de Vigilance météo national',
 'bloc_id': 'BULLETIN_NATIONAL',
 'bloc_items': [{'id': 'NAT_SITUATION_PAYS',
   'type_name': 'Situation Actuelle',
   'type_group': 'SITUATION',
   'text_items': [{'type_code': 'SITUATION_PAYS_SITUATION_GÉNÉRALE_NATIONALE',
     'hazard_code': None,
     'hazard_name': 'tous aléas',
     'term_items': [{'term_names': 'J+J1',
       'start_time': '2024-04-08T08:00:00Z',
       'end_time': '2024-04-09T22:00:00Z',
       'risk_name': 'Orange',
       'risk_code': '3',
       'risk_color': '#f7a401',
       'risk_level': 2,
       'subdivision_text': [{'underline_text': '',
         'text': ['Faits nouveaux : ',
          'Néant pour les paramètres météorologiques.',
          'Vigilance orange "Crues", consultez "Vigicrues" (https://www.vigicrues.gouv.fr/).',
          '',
          '',
          'Situation générale :',
          'La dépression nommée PIERRICK à 990 hPa se rapproche de l

In [72]:
data['product']['text_bloc_items']

[{'domain_id': 'FRA',
  'domain_name': 'France',
  'bloc_title': 'Bulletin de Vigilance météo national',
  'bloc_id': 'BULLETIN_NATIONAL',
  'bloc_items': [{'id': 'NAT_SITUATION_PAYS',
    'type_name': 'Situation Actuelle',
    'type_group': 'SITUATION',
    'text_items': [{'type_code': 'SITUATION_PAYS_SITUATION_GÉNÉRALE_NATIONALE',
      'hazard_code': None,
      'hazard_name': 'tous aléas',
      'term_items': [{'term_names': 'J+J1',
        'start_time': '2024-04-08T08:00:00Z',
        'end_time': '2024-04-09T22:00:00Z',
        'risk_name': 'Orange',
        'risk_code': '3',
        'risk_color': '#f7a401',
        'risk_level': 2,
        'subdivision_text': [{'underline_text': '',
          'text': ['Faits nouveaux : ',
           'Néant pour les paramètres météorologiques.',
           'Vigilance orange "Crues", consultez "Vigicrues" (https://www.vigicrues.gouv.fr/).',
           '',
           '',
           'Situation générale :',
           'La dépression nommée PIERRICK à 

In [73]:
import pandas as pd

In [85]:
data['product']['text_bloc_items']

[{'domain_id': 'FRA',
  'domain_name': 'France',
  'bloc_title': 'Bulletin de Vigilance météo national',
  'bloc_id': 'BULLETIN_NATIONAL',
  'bloc_items': [{'id': 'NAT_SITUATION_PAYS',
    'type_name': 'Situation Actuelle',
    'type_group': 'SITUATION',
    'text_items': [{'type_code': 'SITUATION_PAYS_SITUATION_GÉNÉRALE_NATIONALE',
      'hazard_code': None,
      'hazard_name': 'tous aléas',
      'term_items': [{'term_names': 'J+J1',
        'start_time': '2024-04-08T08:00:00Z',
        'end_time': '2024-04-09T22:00:00Z',
        'risk_name': 'Orange',
        'risk_code': '3',
        'risk_color': '#f7a401',
        'risk_level': 2,
        'subdivision_text': [{'underline_text': '',
          'text': ['Faits nouveaux : ',
           'Néant pour les paramètres météorologiques.',
           'Vigilance orange "Crues", consultez "Vigicrues" (https://www.vigicrues.gouv.fr/).',
           '',
           '',
           'Situation générale :',
           'La dépression nommée PIERRICK à 

In [88]:
[region for region in data['product']['text_bloc_items'] if len(region['bloc_items']) != 0 ]

[{'domain_id': 'FRA',
  'domain_name': 'France',
  'bloc_title': 'Bulletin de Vigilance météo national',
  'bloc_id': 'BULLETIN_NATIONAL',
  'bloc_items': [{'id': 'NAT_SITUATION_PAYS',
    'type_name': 'Situation Actuelle',
    'type_group': 'SITUATION',
    'text_items': [{'type_code': 'SITUATION_PAYS_SITUATION_GÉNÉRALE_NATIONALE',
      'hazard_code': None,
      'hazard_name': 'tous aléas',
      'term_items': [{'term_names': 'J+J1',
        'start_time': '2024-04-08T08:00:00Z',
        'end_time': '2024-04-09T22:00:00Z',
        'risk_name': 'Orange',
        'risk_code': '3',
        'risk_color': '#f7a401',
        'risk_level': 2,
        'subdivision_text': [{'underline_text': '',
          'text': ['Faits nouveaux : ',
           'Néant pour les paramètres météorologiques.',
           'Vigilance orange "Crues", consultez "Vigicrues" (https://www.vigicrues.gouv.fr/).',
           '',
           '',
           'Situation générale :',
           'La dépression nommée PIERRICK à 

In [118]:
infos = [region['bloc_items'] for region in data['product']['text_bloc_items'] if len(region['bloc_items']) != 0 ]

In [119]:
infos

[[{'id': 'NAT_SITUATION_PAYS',
   'type_name': 'Situation Actuelle',
   'type_group': 'SITUATION',
   'text_items': [{'type_code': 'SITUATION_PAYS_SITUATION_GÉNÉRALE_NATIONALE',
     'hazard_code': None,
     'hazard_name': 'tous aléas',
     'term_items': [{'term_names': 'J+J1',
       'start_time': '2024-04-08T08:00:00Z',
       'end_time': '2024-04-09T22:00:00Z',
       'risk_name': 'Orange',
       'risk_code': '3',
       'risk_color': '#f7a401',
       'risk_level': 2,
       'subdivision_text': [{'underline_text': '',
         'text': ['Faits nouveaux : ',
          'Néant pour les paramètres météorologiques.',
          'Vigilance orange "Crues", consultez "Vigicrues" (https://www.vigicrues.gouv.fr/).',
          '',
          '',
          'Situation générale :',
          'La dépression nommée PIERRICK à 990 hPa se rapproche de la Bretagne. Le vent commence à se renforcer sur le littoral du Finistère.',
          '',
          'Observations notables :',
          "Les évèneme

In [89]:
pd.json_normalize([region for region in data['product']['text_bloc_items'] if len(region['bloc_items']) != 0 ])

,domain_id,domain_name,bloc_title,bloc_id,bloc_items
0,FRA,France,Bulletin de Vigilance météo national,BULLETIN_NATIONAL,"[{'id': 'NAT_SITUATION_PAYS', 'type_name': 'Si..."
1,ZDF_OUEST,Défense Ouest,Bulletin de Vigilance météo zonal,BULLETIN_ZONAL,"[{'id': 'ZON_SITUATION_ZONAL', 'type_name': 'S..."
2,14,Calvados,Bulletin de Vigilance météo zone Ouest*,BULLETIN_DEPARTEMENTAL,"[{'id': 'DEP_SITUATION_ZONAL', 'type_name': 'S..."
3,18,Cher,Bulletin de Vigilance météo zone Ouest*,BULLETIN_DEPARTEMENTAL,"[{'id': 'DEP_SITUATION_ZONAL', 'type_name': 'S..."
4,22,Côtes-d'Armor,Bulletin de Vigilance météo zone Ouest*,BULLETIN_DEPARTEMENTAL,"[{'id': 'DEP_SITUATION_ZONAL', 'type_name': 'S..."
5,27,Eure,Bulletin de Vigilance météo zone Ouest*,BULLETIN_DEPARTEMENTAL,"[{'id': 'DEP_SITUATION_ZONAL', 'type_name': 'S..."
6,28,Eure-et-Loir,Bulletin de Vigilance météo zone Ouest*,BULLETIN_DEPARTEMENTAL,"[{'id': 'DEP_SITUATION_ZONAL', 'type_name': 'S..."
7,29,Finistère,Bulletin de Vigilance météo zone Ouest*,BULLETIN_DEPARTEMENTAL,"[{'id': 'DEP_SITUATION_ZONAL', 'type_name': 'S..."
8,35,Ille-et-Vilaine,Bulletin de Vigilance météo zone Ouest*,BULLETIN_DEPARTEMENTAL,"[{'id': 'DEP_SITUATION_ZONAL', 'type_name': 'S..."
9,36,Indre,Bulletin de Vigilance météo zone Ouest*,BULLETIN_DEPARTEMENTAL,"[{'id': 'DEP_SITUATION_ZONAL', 'type_name': 'S..."


In [51]:
template = """
Voici les informations concernant les vigilences du jour peux tu me faire un résumé en français : {data}
"""

In [52]:
prompt = PromptTemplate.from_template(template)


In [53]:
chain = {'data':RunnablePassthrough()}| prompt | gpt_turbo

In [115]:
chain.invoke(str(infos))

'Aujourd\'hui, il y a une vigilance orange pour les "Crues" dans la région de Bretagne, il est recommandé de consulter le site "Vigicrues" pour plus d\'informations. Une dépression nommée PIERRICK à 990 hPa se rapproche de la région et les vents commencent à se renforcer sur le littoral du Finistère. Il y a également une vigilance orange pour les vents violents dans la région de Finistère à partir de 16h ce lundi, en raison de la dépression qui se déplace le long des côtes sud de l\'Angleterre, entraînant un risque de vagues-submersion. Pour l\'instant, aucun événement notable n\'a été signalé. '